In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
%load_ext autoreload
%autoreload 2

In [ ]:
def standardize(tx):
    mean = np.mean(tx, axis=0)
    std = np.std(tx, axis=0)
    tx = (tx-mean)/std
    return tx

# function that add new features 
def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=1 up to j=degree."""
    phi=np.zeros((x.shape[0],(degree+1)*x.shape[1]))
    for j in range(degree+1):
            phi[:,j*x.shape[1]:(j+1)*x.shape[1]]=x**j
    return phi

def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

def cross_validation_LR(y, x, k_fold, initial_w, max_iters, gamma, seed=1):
    """return the loss of ridge regression."""
    loss_tr = [] 
    loss_te = []
    ws = []
    k_indices = build_k_indices(y, k_fold, seed)
    for k in range(k_fold):
        # ***************************************************
        # get k'th subgroup in test, others in train
        # ***************************************************
        idx_tr = (np.delete(k_indices, k, 0)).flatten()
        idx_te = k_indices[k]
        x_tr, y_tr = x[idx_tr], y[idx_tr]
        x_te, y_te = x[idx_te], y[idx_te]
        # ***************************************************
        # calculate the loss for train and test data
        # ***************************************************
        w, loss = logistic_regression(y_tr, x_tr, initial_w, max_iters, gamma)
        loss_tr.append(loss)
        loss_te.append(logistic_loss(y_te, x_te, w))
        ws.append(w)
    var_tr = np.var(loss_tr)
    var_te = np.var(loss_te)
    loss_tr = np.mean(loss_tr)
    loss_te = np.mean(loss_te)
    ws = np.mean(np.asarray(ws), axis=0)
    return loss_tr, loss_te, var_tr, var_te, ws


def cross_validation_RLR(y, x, k_fold, lambda_, initial_w, max_iters, gamma, seed=1):
    """return the loss of ridge regression."""
    loss_tr = [] 
    loss_te = []
    ws = []
    k_indices = build_k_indices(y, k_fold, seed)
    for k in range(k_fold):
        # ***************************************************
        # get k'th subgroup in test, others in train
        # ***************************************************
        idx_tr = (np.delete(k_indices, k, 0)).flatten()
        idx_te = k_indices[k]
        x_tr, y_tr = x[idx_tr], y[idx_tr]
        x_te, y_te = x[idx_te], y[idx_te]
        # ***************************************************
        # calculate the loss for train and test data
        # ***************************************************
        w, loss = reg_logistic_regression(y_tr, x_tr, lambda_, initial_w, max_iters, gamma)
        loss_tr.append(loss)
        loss_te.append(logistic_loss(y_te, x_te, w) + 0.5*lambda_*w.dot(w))
        ws.append(w)
    var_tr = np.var(loss_tr)
    var_te = np.var(loss_te)
    loss_tr = np.mean(loss_tr)
    loss_te = np.mean(loss_te)
    ws = np.mean(np.asarray(ws), axis=0)
    return loss_tr, loss_te, var_tr, var_te, ws


# First : exploring the data

We'll need to have a look at what the data is, how it is distributed for the different features, and start to get an intuition about what methods might work better for analysis and prediction later.

## Load the training data into feature matrix, class labels, and event ids:

In [ ]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [ ]:
# testing methods work
# initial_w = np.ones(tX.shape[1])
# max_iters = 100
# gamma = 0.1
# lambda_ = 0.1
# print('least_squares_GD')
# print(least_squares_GD(y, tX, initial_w, max_iters, gamma))
# print('least_squares_SGD')
# print(least_squares_SGD(y, tX, initial_w, max_iters, gamma))
# print('least_squares')
# print(least_squares(y, tX))
# print('ridge_regression')
# print(ridge_regression(y, tX, lambda_))
# print('logistic_regression SGD')
# print(logistic_regression(y, tX, initial_w, max_iters, gamma, 'SGD'))
# print('logistic_regression GD')
# print(logistic_regression(y, tX, initial_w, max_iters, gamma, 'GD'))
# print('reg_logistic_regression SGD')
# print(reg_logistic_regression(y, tX, lambda_, initial_w, max_iters, gamma, 'SGD'))
# print('reg_logistic_regression GD')
# # print(reg_logistic_regression(y, tX, lambda_, initial_w, max_iters, gamma, 'GD'))
# print('logistic_regression_newton')
# print(logistic_regression_newton(y[:100], tX[:100], initial_w, max_iters, gamma))
# print('reg_logistic_regression_newton')
# print(reg_logistic_regression_newton(y[:100], tX[:100], lambda_, initial_w, max_iters, gamma))

In [ ]:
# remove samples with error values
idx_c = np.all(tX!=-999, axis=1)
y_c = y[idx_c]
tX_c = tX[idx_c]
# regularize
mean = np.mean(tX_c, axis=0)
std = np.std(tX_c, axis=0)
tX_c = (tX_c-mean)/std

In [ ]:
print("Overall: s: ",np.sum(y==1),", b: ",np.sum(y==-1)," ,total:",len(y))
print("NoErrors: s: ",np.sum(y_c==1),", b: ",np.sum(y_c==-1)," ,total:",len(y_c))
for n in range(tX_c.shape[1]):
    plt.figure(figsize=(20,4))
    plt.subplot(131)
    plt.hist([tX_c[y_c==1,n],tX_c[y_c==-1,n]], 20, density=True, histtype='bar', stacked=True)
    plt.legend(['s','b'])
    plt.title('Feature '+str(n))
    plt.subplot(132)
    plt.title('s histogram feature '+str(n))
    plt.hist(tX_c[y_c==1,n], 20, density=True, histtype='bar', stacked=True)
    plt.subplot(133)
    plt.title('b histogram feature '+str(n))
    plt.hist(tX_c[y_c==-1,n], 20, density=True, histtype='bar', stacked=True)    
    plt.show()

In [ ]:
# remove features with error values
idx_gf = np.arange(tX.shape[1])[np.all(tX!=-999, axis=0)]
y_gf = y
tX_gf = tX[:,idx_gf]
# regularize
mean = np.mean(tX_gf, axis=0)
std = np.std(tX_gf, axis=0)
tX_gf = (tX_gf-mean)/std

In [ ]:
print("Overall: s: ",np.sum(y==1),", b: ",np.sum(y==-1)," ,total:",len(y))
for n in range(tX_gf.shape[1]):
    plt.figure(figsize=(20,4))
    plt.subplot(131)
    plt.hist([tX_gf[y_gf==1,n],tX_gf[y_gf==-1,n]], 20, density=True, histtype='bar', stacked=True)
    plt.legend(['s','b'])
    plt.title('Feature '+str(idx_gf[n]))
    plt.subplot(132)
    plt.title('s histogram feature '+str(idx_gf[n]))
    plt.hist(tX_gf[y_gf==1,n], 20, density=True, histtype='bar', stacked=True)
    plt.subplot(133)
    plt.title('b histogram feature '+str(idx_gf[n]))
    plt.hist(tX_gf[y_gf==-1,n], 20, density=True, histtype='bar', stacked=True)    
    plt.show()

In [ ]:
# remove features with error values
y_jet = []
tx_jet = []
y_jet_nm = []
tx_jet_nm = []
# filtering according to undefinition due to jet number
idx_jet_undef = [np.array([0,1,2,3,7,10,11,13,14,15,16,17,18,19,20,21,29]),
                np.array([0,1,2,3,7,8,9,10,11,13,14,15,16,17,18,19,20,21,23,24,25,29]),
                np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,23,24,25,26,27,28,29]),
                np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,23,24,25,26,27,28,29])]
# Extra filtering according to definition of mass
idx_jet_undef_nm = [np.array([1,2,3,7,10,11,13,14,15,16,17,18,19,20,21,29]),
                    np.array([1,2,3,7,8,9,10,11,13,14,15,16,17,18,19,20,21,23,24,25,29]),
                    np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,23,24,25,26,27,28,29]),
                    np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,23,24,25,26,27,28,29])]
for jet in range(4):
    idx_jet = tX[:,22]==jet
    y_jet.append(y[idx_jet])
    tx_jet.append(tX[idx_jet][:,idx_jet_undef[jet]])
    # tx_jet.append(standardize(tX[idx_jet]))
for jet in range(4):
    idx_jet = tX[:,22]==jet
    y_jet_nm.append(y[idx_jet])
    tx_jet_nm.append(tX[idx_jet][:,idx_jet_undef_nm[jet]])
    # tx_jet.append(standardize(tX[idx_jet]))

for jet in range(4):
    print('Jet {:} shape is {:}'.format(jet,tx_jet[jet].shape))

In [ ]:
print("Overall: s: ",np.sum(y==1),", b: ",np.sum(y==-1)," ,total:",len(y))
for jet in range(4):
    print('Jet {:}: s: {:}, b: {:} ,total: {:}'.format(jet, np.sum(y_jet[jet]==1),np.sum(y_jet[jet]==-1),len(y_jet[jet])))
    for n,feat in enumerate(idx_jet_undef[jet]):
        plt.figure(figsize=(20,4))
        plt.subplot(131)
        plt.hist([tx_jet[jet][y_jet[jet]==1,n],tx_jet[jet][y_jet[jet]==-1,n]], 20, density=True, histtype='bar', stacked=True)
        plt.legend(['s','b'])
        plt.title('Feature '+str(feat))
        plt.subplot(132)
        plt.title('s histogram feature '+str(feat))
        plt.hist(tx_jet[jet][y_jet[jet]==1,n], 20, density=True, histtype='bar', stacked=True)
        plt.subplot(133)
        plt.title('b histogram feature '+str(feat))
        plt.hist(tx_jet[jet][y_jet[jet]==-1,n], 20, density=True, histtype='bar', stacked=True)    
        plt.show()

In [ ]:
print("Overall: s: ",np.sum(y==1),", b: ",np.sum(y==-1)," ,total:",len(y))
for jet in range(4):
    print('Jet {:}: s: {:}, b: {:} ,total: {:}'.format(jet, np.sum(y_jet_nm[jet]==1),np.sum(y_jet_nm[jet]==-1),len(y_jet_nm[jet])))
    for n,feat in enumerate(idx_jet_undef_nm[jet]):
        plt.figure(figsize=(20,4))
        plt.subplot(131)
        plt.hist([tx_jet_nm[jet][y_jet_nm[jet]==1,n],tx_jet_nm[jet][y_jet_nm[jet]==-1,n]], 20, density=True, histtype='bar', stacked=True)
        plt.legend(['s','b'])
        plt.title('Feature '+str(feat))
        plt.subplot(132)
        plt.title('s histogram feature '+str(feat))
        plt.hist(tx_jet_nm[jet][y_jet_nm[jet]==1,n], 20, density=True, histtype='bar', stacked=True)
        plt.subplot(133)
        plt.title('b histogram feature '+str(feat))
        plt.hist(tx_jet_nm[jet][y_jet_nm[jet]==-1,n], 20, density=True, histtype='bar', stacked=True)    
        plt.show()

# Actual predictions start from here

After having looked at the data we will now do some actual predictions using different models andd parameters.

In [ ]:
tx_full = tx_jet+tx_jet_nm
y_full = y_jet+y_jet_nm
# for y_i in y_full:
#     y_i[y_i==-1] = 0
max_iters = 1000
k_fold = 5

In [ ]:
# Logistic regression
gammas = np.geomspace(0.001,2,15)
loss_tr = np.zeros((8,len(gammas)))
loss_te = np.zeros((8,len(gammas)))
var_tr = np.zeros((8,len(gammas)))
var_te = np.zeros((8,len(gammas)))
for i in range(8):
    tx_i = tx_full[i]
    y_i = y_full[i]
    for g,gamma in enumerate(gammas):
        initial_w = np.zeros(tx_i.shape[1])
        loss_tr[i,g],loss_te[i,g], var_tr[i,g], var_te[i,g], ws = cross_validation_LR(y_i, tx_i, k_fold, initial_w, max_iters, gamma)
        print('set {:} - train_loss: {:}, test_loss: {:}, train_var: {:}, test_var: {:}'.format(i,loss_tr[i,g],loss_te[i,g], var_tr[i,g], var_te[i,g]))
        print(ws)

In [ ]:
# Regularized logistic regression
# gammas = np.geomspace(1e-8,1e-5,8)
gamma = 1e-9
lambdas = np.geomspace(1e-4,1,5)
seeds = np.arange(100)
# loss_tr = np.zeros((8,len(lambdas),len(gammas)))
# loss_te = np.zeros((8,len(lambdas),len(gammas)))
# var_tr = np.zeros((8,len(lambdas),len(gammas)))
# var_te = np.zeros((8,len(lambdas),len(gammas)))
loss_tr = np.zeros((8,len(seeds),len(lambdas)))
loss_te = np.zeros((8,len(seeds),len(lambdas)))
var_tr = np.zeros((8,len(seeds),len(lambdas)))
var_te = np.zeros((8,len(seeds),len(lambdas)))
for i in range(8):
    tx_i = tx_full[i]
    y_i = y_full[i]
    for l,lambda_ in enumerate(lambdas):
#         for g,gamma in enumerate(gammas):
        for s,seed in enumerate(seeds):
            initial_w = np.zeros(tx_i.shape[1])
    #         loss_tr[i,l,g],loss_te[i,l,g], var_tr[i,l,g], var_te[i,l,g], ws = cross_validation_RLR(y_i, tx_i, k_fold, lambda_, initial_w, max_iters, gamma)
    #         print('set {:} lambda {:} gamma {:} -\n train_loss: {:.6f}, test_loss: {:.6f}, train_var: {:.6f}, test_var: {:.6f}'.format(i,lambda_,gamma,loss_tr[i,l,g],loss_te[i,l,g], var_tr[i,l,g], var_te[i,l,g]))
            loss_tr[i,s,l],loss_te[i,s,l], var_tr[i,s,l], var_te[i,s,l], ws = cross_validation_RLR(y_i, tx_i, k_fold, lambda_, initial_w, max_iters, gamma, seed)
            print('set {:} lambda {:}-\n train_loss: {:.6f}, test_loss: {:.6f}, train_var: {:.6f}, test_var: {:.6f}'.format(i,lambda_,loss_tr[i,s,l],loss_te[i,s,l], var_tr[i,s,l], var_te[i,s,l]))
    #         print(ws)

In [ ]:
for i in range(8):
    print('Average Loss for set {} is: train: {}, test: {}'.format(i,np.mean(loss_tr[i]),np.mean(loss_te[i])))

Average Loss for set 0 is: train: 54980.74763044169, test: 13745.20416065596<br>
Average Loss for set 1 is: train: 40983.29810703878, test: 10245.85299564541<br>
Average Loss for set 2 is: train: 21006.93861075539, test: 5251.771478499431<br>
Average Loss for set 3 is: train: 10771.50148604793, test: 2692.925730373346<br>
Average Loss for set 4 is: train: 55011.27157849578, test: 13752.82139825698<br>
Average Loss for set 5 is: train: 41129.31680385559, test: 10282.34753957150<br>
Average Loss for set 6 is: train: 21107.61567660550, test: 5276.935237919042<br>
Average Loss for set 7 is: train: 10792.49529019947, test: 2698.171831750048

In [ ]:
for i,var in enumerate(var_te):
    plt.figure(figsize=(16,6))
    plt.subplot(121)
    for seed in enumerate(seeds):
        plt.plot(loss_te[i,seed], 'b')
    plt.
    plt.title('Loss of Features set '+str(i))
    plt.xlabel('lambda')
    plt.ylabel('error')
    plt.subplot(122)
    for seed in range(len(seeds)):
        plt.plot(var[seed])
        plt.title('Variation of Loss of Features set '+str(i))
        plt.xlabel('lambda')
        plt.ylabel('variation')
    plt.show()

In [ ]:
for i,var in enumerate(var_te):
    plt.figure(figsize=(16,6))
    plt.subplot(121)
    plt.pcolor(loss_te[i])
    plt.colorbar()
    plt.title('Loss of Features set '+str(i))
    plt.xlabel('gamma')
    plt.ylabel('lambda')
    plt.subplot(122)
    plt.pcolor(np.log10(var))
    plt.colorbar()
    plt.title('Variation of Features set '+str(i))
    plt.xlabel('gamma')
    plt.ylabel('lambda')
    plt.show()

## Load the test data

In [ ]:
from proj1_helpers import *
DATA_TEST_PATH = '../data/test.csv.zip' 
# test_y, test_X, ids = load_csv_data(DATA_TRAIN_PATH)

## Generate predictions using only features with no errrors throughought

This enables us to use some of the methods from the course directly, without having to adjust some of the functionnality to account for the fact that a lot of errors are in the dataset. First let us see which features from the test dataset are error free.

In [ ]:
# Import test data

import pandas as pd

test_x_df = pd.read_csv(DATA_TEST_PATH, index_col='Id').drop('Prediction', axis='columns')
test_x_df

train_x_df = pd.read_csv(DATA_TRAIN_PATH, index_col='Id').drop('Prediction', axis='columns')
train_y_df = pd.read_csv(DATA_TRAIN_PATH, index_col='Id', usecols=['Id', 'Prediction'])

In [ ]:
# Remove columns with -999 errors
error_cols_te = np.full(test_x_df.shape[1], False)
for i in range(test_x_df.shape[1]):
#     print(test_x_df.iloc[:,i].values)
#     print(-999.0 in test_x_df.iloc[:,i].values)
    if -999.0 in test_x_df.iloc[:,i].values:
        error_cols_te[i] = True
print(f"There are {error_cols_te.tolist().count(True)} test error columns are which are : \n{error_cols_te}\n")

error_cols_tr = np.full(train_x_df.shape[1], False)
for i in range(train_x_df.shape[1]):
    if -999.0 in train_x_df.iloc[:,i].values:
        error_cols_tr[i] = True
print(f"There are {error_cols_tr.tolist().count(True)} train error columns are which are : \n{error_cols_tr}")

no_error_cols_tr_te = (~(error_cols_tr | error_cols_te)).tolist()
print(f"They have {no_error_cols_tr_te.count(True)} following shared non error columns :\n{no_error_cols_tr_te}")

With this figured out we can now extract the valid columns from test and train data, do some training and testing on data, then generate answers for the test data and submit to aicrowd !

# Save prediction ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)